# 3 Функциональный стиль программирования на Ruby

## Немного о Ruby

### Интерпретируемый язык программирования

Нет никакой компиляции, все работает "на лету". Есть специальная программа - интерпретатор, которая получает строчки из файла с программой. Этот интерпретатор - можно сказать, виртуальная машина, которая преобразует высокоуровневый код на Ruby в последовательность своих (не ассемблерных!) инструкций, которые уже в свою очередь в зависимости от конкретной архитектуры вычислительной системы преобразуются в низкоуровневые "реальные" инструкции процессора.

Это работает **медленнее**, чем компиляция.

К тому же, так как никто не проверяет **все**, что вы написали, перед запуском, программа упадет в первой точке, где произошла ошибка. То есть вы узнаете о первой произошедшей ошибке, а об остальных, если они есть, только после того, как исправите первую.

Например здесь 2 ошибки - использование неинициализированной переменной `c` и вызов несуществующей функции. Если мы попросим интерпретатор выполнить этот скрипт, то увидим информацию только о первой ошибке:

In [1]:
a = 10
b = 20

a += c

a = b + a / 4.0

not_existing_function(a)

NameError: undefined local variable or method `c' for main:Object

Теперь мы ее исправим и получим информацию о второй:

In [3]:
a = 10
b = 20
c = 0

a += c

a = b + a / 4.0

not_existing_function(a)

NoMethodError: undefined method `not_existing_function' for main:Object

Это приводит к тому, что отлаживать большие программы на интерпретируемых языках может быть тяжелее, чем на компилируемых. Но на самом деле все очень сильно зависит от подбора средств разработки и отладки, кучу страданий можно не испытывать. К тому же накидать что-то на Ruby гораздо легче и быстрее, чем на С++/Java/Delphi (!!!).

**Программа исполняется сверху вниз, если только явно не указано обратное (о том, как это сделать, мы поговорим чуть позже).**

### Динамический язык программирования

Красиво звучит, но к чему это приводит в действительности?

Переменная может принимать значение любого типа (**тут аккуратнее с терминологией**)

In [9]:
a = 10
# Печатает класс переменной
puts(a.class())

a = 'string'
a.class()

Fixnum


String

Сначала число, потом строка, все это в одной переменной и никакого противоречия нет, хотя в любом нормальном языке со статической типизацией вам бы это с рук не сошло.

Удобно? Да. Опасно? Еще бы! Можно выстрелить себе в ногу, когда будете писать функции, ведь можно не учесть того, что вам на вход функции сложения двух целых чисел может прийти не то, что одно число с плавающей точкой и одна строка, а вообще ВСЕ, что угодно из того, что есть в языке.

Но не все так плохо: во-первых, есть средства для того, чтобы не пораниться при программировании. К тому же вы увидите, почему динамические языки программирования - это очень крутая и гибкая штука.

### Объектно-ориентированный язык программирования

Все есть объект, до абсурда. То есть даже казалось бы численный литерал совершенно спокойно отдаст вам свой класс, а еще вы сможете делать кучу прикольных штук с ним:

In [10]:
# Печатаем класс числа 10 (!!!)
puts(10.class())

# Можно преобразовать в строку
puts(10.to_s())

# А можно вообще сделать своеобразный цикл
3.times do
  puts('Я в цикле!')
end

Fixnum
10
Я в цикле!
Я в цикле!
Я в цикле!


3

**Более того, даже класс - это объект!**

In [12]:
# Array - это встроенный класс массива. Можно посмотреть, что это объект класса Class
puts(Array.class())

# Более того, этот КЛАСС можно присвоить переменной и передать в качестве аргумента в функцию!
cls = Array
puts(cls.class())
puts(cls)

Class
Class
Array


Это некоторые основы, которые нужно помнить при программировании на Ruby. Сейчас мы посмотрим, что эти основы за собой влекут.

## Последствия

### Переменная - это не переменная

В С++ переменная имеет **жестко заданный тип** и определяет **конкретную область памяти**. В Ruby переменная - это **указатель** на реальную переменную. В первую очередь благодаря этому переменная может быть любого типа, то есть ей можно переприсваивать значения. К тому же все типы данных наследуются от Object, благодаря чему у них есть куча общих методов.

Работает это так. Интерпретатор создает переменные при первом упоминании и связыает с переменной, которая фигурирует в этом упоминании в левой части выражения. Интерпретатор же освобождает эти переменные когда посчитает нужным, то есть когда они перестают использоваться программистом. А переменная в программе - это просто ссылка на реально создаваемую интерпретаторм переменную.

In [14]:
a = 10
b = 10

a += 1

puts(a, b)

11
10


Тут вроде все логично, как и ожидалось.

In [15]:
a = 10
b = a

a += 1

puts(a, b)

11
10


Тут интерпретатор тоже оказался умнее нас, все учел.

In [18]:
a = [1, 2, 3]
b = a

a += [4]

puts(a, b)

[1, 2, 3, 4]
[1, 2, 3]


И тут все логично. НО

In [1]:
a = [1, 2, 3]
b = a

a << 4

puts(b)

[1, 2, 3, 4]
21


21

Коллизия! Такого быть не должно. Так почему же где-то все работает согласно логике, а где-то нет? Некоторые методы создают **новые** объекты, например, += создает новый список, а некоторые работают с текущим, например << (это вставка значения). Иными словами, когда мы пишем a += b, то а начинает ссылаться на **новый** объект. А в последнем примере нового объекта не создается.

**Если программа творит дичь, проверьте наличие таких коллизий!**

## Немного синтаксиса

### Ruby - один сплошной синтаксический сахар

Куча синонимов для одного и того же действия, некоторые приятные особенности синтаксиса и все это ради читаемости кода!

### Скобки при вызове методов и функций необязательны

In [23]:
# Найдите 228 отличий
puts('228')
puts '228'

228
228


По методам вывода:

p - отладочная печать

puts - печатает красиво

print - печатает без переноса на следующую строку

### Неизменяемые строки - символы

In [27]:
# Вот это просто строка
str = 'string'
p str.class

# Мы можем сделать так
str[1] = 'T'

p str

# Вот это неизменяемая строка
smb = :symbol
p smb.class

# Так не прокатит
smb[1] = 'Y'

String
"sTring"
Symbol


NoMethodError: undefined method `[]=' for :symbol:Symbol
Did you mean?  []

In [30]:
# Строку можно привести к символу (сделать неизменяемой) и наоборот
p str.to_sym.class
p smb.to_s.class

Symbol
String


String

### Не обязательно писать return

Из функции вернется **результат последнего выражения**

### Цепочки методов и многое-многое другое...

## Примеры создания проекта

Заглушка - ex1

## Как нужно делать в лабораторных и почему

Заглушка - ex2

### Как улучшить функцию

In [5]:
def square(x)
  x**2 if x.is_a? Numeric or nil
end

p square(2)
p square(2.7)
p square('fjfj')

4
7.290000000000001
nil


## Задачи

Вычислить значение выражения $$a = {{y - x^2} \over {x - {y} \over {b + x^2}}}$$

Допустим, переменные мы уже ввели и нам осталось только сконструировать выражение.

In [6]:
b = 11
x = 5
y = 10

(y - x**2) / ((x - y) / (b + x**2))

15

Неправильно же! Почему?

In [7]:
10 / 3

3

In [7]:
10 / 3.0

3.3333333333333335

В этом и кроется причина

In [9]:
b = 11.0
x = 5.0
y = 10.0

(y - x**2) / ((x - y) / (b + x**2))

108.0

Почти готово, попробуем перенести в функцию

In [11]:
def calculate(b, x, y)
  (y - x**2) / ((x - y) / (b + x**2))
end

:calculate

In [12]:
calculate 11, 5, 10

15

Как быть с преобразованием во float в функции??? Например, так

In [13]:
def calculate(b, x, y)
  b, x, y = b.to_f, x.to_f, y.to_f
  (y - x**2) / ((x - y) / (b + x**2))
end

:calculate

In [14]:
calculate 11, 5, 10

108.0

Ок, что делать с примерами посложнее? Курить модуль [Math](http://ruby-doc.org/core-1.9.3/Math.html)

Решим пример из лабы $$|tg{{x^2 \cdot (x - 2)} \over {e^x}}|$$

In [16]:
include Math

x = 10.to_f

Math.tan(x**2 * (x - 2) / Math.exp(x)).abs

0.0363359225845265

## Функциональный стиль

Что это? Некоторое подражание [функциоальному программированию](https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5). Но, конечно, больше, чем просто подражание.

1) часто это удобно - кратко и емко;

2) это хорошо читается (правда, нужно немного привыкнуть);

3) это **идеоматично** - Ruby оптимизирован под такой стиль.

Сравните

In [22]:
a = [1, 2, 3]

for i in 0...a.length
  puts a[i]
end

1
2
3


0...3

In [21]:
a = [1, 2, 3]

a.each do |x|
  puts x
end

1
2
3


[1, 2, 3]

In [24]:
# И даже более того

[1, 2, 3].each do |x|
  puts x
end

1
2
3


[1, 2, 3]

Можно даже сказать, что функциональный стиль быстрее императивного.

In [33]:
n_elems = 1_000_000
n_iterations = 20 

a = Array.new(n_elems, 1)

overall_time = 0

n_iterations.times do
  start = Time.now
  for i in 0...a.length
    a[i] = a[i]**2
  end
  overall_time += Time.now - start
end

overall_time.to_f / n_iterations.to_f

0.23686489785

In [34]:
a = Array.new(n_elems, 1)

overall_time = 0

n_iterations.times do
  start = Time.now
  a.map {|x| x**2} # Функциональщина в этой строчке
  overall_time += Time.now - start
end

overall_time.to_f / n_iterations.to_f

0.08003850994999999

### Что нужно, чтобы научиться прогать в рамках функционального стиля

### Изучить методы классов - Array, String, Hash

### Array - массив необязательно однородных элементов

In [36]:
a = Array.new(10, 1)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Срез массива

In [32]:
a = [1, 1, 1, 2, 2, 1]
a[3..4]

[2, 2]

### String - строка

In [34]:
s = 'fl;m'
s[0..1]

"fl"

### Hash - хэш-таблица, словарь, хэш-мап

In [36]:
h = {:symbol1 => 'value1', :symbol2 => 'value2'}
# или
h = {symbol1: 'value1', symbol2: 'value2'}

{:symbol1=>"value1", :symbol2=>"value2"}

Как раз тут нужны неизменяемые стоки, потому что если строка изменится, хэш изменится тоже.

In [37]:
h = {'fff': 'fff'} # будет неявное преобразование в symbol

{:fff=>"fff"}

**Немного функциональщины**

In [8]:
a = Array.new(10, 1)
p a

a.map! {|x| x * 3}
a

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

## Задачи

### 1 Проверить, является ли строка палиндромом

Строка палиндром тогда, когда ее чтение с начала и с конца дает одинаковый результат.

Пример: А роза упала на лапу Азора.

**Примечание** Регистр букв мы здесь учитывать не будем.

In [40]:
def palindrome?(string)
  # TODO
end

ex = 'А роза упала на лапу Азора'

palindrome? ex

In [5]:
# У строки есть метод reverse, он возвращает развернутую в обратном порядке копию строки.
# Попробуем воспользоваться им.

p ex == ex.reverse

# Почему нет?

p ex.reverse

# Мешается регистр символов + пробелы!

p ex.downcase
p ex.delete ' '

p ex.downcase.delete ' '

false
"арозА упал ан алапу азор А"
"А роза упала на лапу Азора"
"АрозаупаланалапуАзора"
"АрозаупаланалапуАзора"


"АрозаупаланалапуАзора"

Downcase не работает!

`gem install unicode_utils`

In [2]:
require 'unicode_utils'

UnicodeUtils.downcase 'БББ'

"ббб"

In [39]:
def palindrome?(ex)
  UnicodeUtils.downcase(ex.delete! ' ') == UnicodeUtils.downcase(ex.reverse)
end

palindrome? 'А роза упала на лапу Азора'

true

### 2 Посчитать сумму квадратов элементов массива

In [8]:
a = [1, 3, 7, 9]

a.inject(0) {|sum, x| sum += x**2}

140

### 3.1 Пример со строками

Дана последовательность строк. Каждая строка состоит из слов, разделенных пробелами. 
Написать программу, обеспечивающую ввод строк и их корректировку. Корректировка 
заключается в удалении лишних пробелов и слов, состоящих из одного 
символа. 
Лишними считаются пробелы в начале и конце строки, а также более одного пробела 
между словами. Вывести на печать исходную и скорректированную последовательности 
строк.

In [21]:
string = '    sfk sfg   flg s        l    '

p string.split # пробел можно не писать, это аргумент по умолчанию.

string.split.reject {|x| x.length == 1}.join ' '

["sfk", "sfg", "flg", "s", "l"]


"sfk sfg flg"

### 3.2 Еще один пример со строками

Дана последовательность строк. Каждая строка состоит из слов, разделенных пробелами. Написать программу, обеспечивающую ввод строк
и их корректировку. Корректировка заключается в удалении или замене
слов. Если слово стоит на четном месте и начинается на букву «е» — оно
удаляется; если слово стоит на четном месте и начинается на букву «с»,
оно замещается на слово, введенное с клавиатуры. Вывести на печать
исходную и скорректированную последовательности строк.

In [15]:
word_to_replace = 'replaced'

remove_letter = 'e'
replace_letter = 'c'

string = '  fgm    dfkm fdkbdmfb lfdb      eremove fff creplace '  

word_array = string.split.map.with_index do |x, idx|
  if idx.even? and x.start_with? remove_letter
    nil
  elsif idx.even? and x.start_with? replace_letter
    word_to_replace
  else
    x
  end
end

# compact удаляет nil
word_array.compact.join ' '

"fgm dfkm fdkbdmfb lfdb fff replaced"

### 3 Найти все элементы массива, кратные трем, и вывести их с индексом, а также сумму их индексов

In [43]:
a = [1, 3, 7, 9]

a.reject {|x| 0 != x % 3}.each_with_index {|x, idx| puts "#{x}: #{idx}"}.each_index.inject(0) {|sum, idx| sum += idx}

3: 0
9: 1


1